In [1]:
from qiskit import QuantumCircuit
from qiskit.dagcircuit import DAGCircuit
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.tools.visualization import dag_drawer
import pickle

from SparsePauliExperiment import LearningProcedure

In [2]:
qc = QuantumCircuit(4,4)
qc.x(0)
qc.cx(0,1)
qc.cx(2,3)
qc.x(0)
qc.x(1)
qc.cx(0,1)
qc.cx(2,3)
qc.cx(1,2)
qc.measure(range(4), range(4))

In [3]:
qc.draw()

┌───┐     ┌───┐          ┌─┐      
q_0: ┤ X ├──■──┤ X ├──■───────┤M├──────
     └───┘┌─┴─┐├───┤┌─┴─┐     └╥┘┌─┐   
q_1: ─────┤ X ├┤ X ├┤ X ├──■───╫─┤M├───
          └───┘└───┘└───┘┌─┴─┐ ║ └╥┘┌─┐
q_2: ──■────■────────────┤ X ├─╫──╫─┤M├
     ┌─┴─┐┌─┴─┐ ┌─┐      └───┘ ║  ║ └╥┘
q_3: ┤ X ├┤ X ├─┤M├────────────╫──╫──╫─
     └───┘└───┘ └╥┘            ║  ║  ║ 
c: 4/════════════╩═════════════╩══╩══╩═
                 3             0  1  2

In [8]:
layers, measurements = circuit_to_benchmark_layers(qc)
render_layers(layers, measurements).draw()

┌───┐      ░ ┌───┐      ░       ░ ┌─┐         
q_0: ┤ X ├──■───░─┤ X ├──■───░───────░─┤M├─────────
     └───┘┌─┴─┐ ░ ├───┤┌─┴─┐ ░       ░ └╥┘┌─┐      
q_1: ─────┤ X ├─░─┤ X ├┤ X ├─░───■───░──╫─┤M├──────
          └───┘ ░ └───┘└───┘ ░ ┌─┴─┐ ░  ║ └╥┘┌─┐   
q_2: ──■────────░───■────────░─┤ X ├─░──╫──╫─┤M├───
     ┌─┴─┐      ░ ┌─┴─┐      ░ └───┘ ░  ║  ║ └╥┘┌─┐
q_3: ┤ X ├──────░─┤ X ├──────░───────░──╫──╫──╫─┤M├
     └───┘      ░ └───┘      ░       ░  ║  ║  ║ └╥┘
c: 4/═══════════════════════════════════╩══╩══╩══╩═
                                        0  1  2  3

In [5]:
def get_measurements(qc):
    measurements = qc.copy_empty_like()
    for inst in qc:
        if inst.operation.name == "measure":
            measurements.append(inst)
    return measurements

def circuit_to_benchmark_layers(qc):
    layers = []
    #remove final measurements and store as separate layer
    measurements = get_measurements(qc)
    #convert qc to a list of instructions
    inst_list = [inst for inst in qc if inst.operation.name != "measure"] 
    #iterate through instructions adding to layers according to the rules until all are added
    while inst_list:
        circ = qc.copy_empty_like()
        layer_qubits = set()

        for inst in inst_list.copy():
            if not layer_qubits.intersection(inst.qubits):
                if len(inst.qubits) == 2:
                    layer_qubits = layer_qubits.union(inst.qubits)
                circ.append(inst)
                inst_list.remove(inst)

        layers.append(circ)

    return layers, measurements

In [7]:
def render_layers(layers, measurements):
    qc = layers[0].copy_empty_like()
    for l in layers:
        qc = qc.compose(l)
        qc.barrier()
    qc = qc.compose(measurements)
    return qc

In [9]:
def hashable_layer(layer):
    return tuple((inst.operation.name,) + tuple(q.index for q in inst.qubits) for inst in layer if len(inst.qubits) == 2)

In [10]:
def benchmark_profiles(layers):
    return list(dict([(hashable_layer(layer),layer) for layer in layers]).values())

In [11]:
benchmark_profiles(layers)

/tmp/ipykernel_26408/174004657.py:2: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  return tuple((inst.operation.name,) + tuple(q.index for q in inst.qubits) for inst in layer if len(inst.qubits) == 2)


In [12]:
def benchmark_experiment(circuit, save_dir, backend, inst_map = None, depths = [1,4,16], samples = 50, single_samples = 50):
    with open(save_dir+"/circuit.pickle", "wb") as f:
        pickle.dump(circuit, f)
    layers,_ = circuit_to_benchmark_layers(circuit)
    profiles = benchmark_profiles(layers)

    if not inst_map:
        inst_map = sorted([q.index for q in circuit.qubits])

    for i,profile in enumerate(profiles):
        procedure = LearningProcedure(profile, inst_map, backend)
        data = {"layer":profile, "inst_map": inst_map, "samples":samples, "single_samples":single_samples, "model_terms": procedure.model_terms}
        circs = procedure.procedure(depths, samples, single_samples)
        
        with open(save_dir+"/Profile"+str(i)+".data", 'wb') as f:
            pickle.dump(data, f)

        with open(save_dir+"/Profile"+str(i)+".circs", 'wb') as f:
            pickle.dump(circs, f)

In [13]:
from qiskit.providers.fake_provider import FakeVigoV2

In [14]:
benchmark_experiment(qc, "Data", FakeVigoV2(), samples = 10, single_samples = 10)

/tmp/ipykernel_26408/174004657.py:2: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  return tuple((inst.operation.name,) + tuple(q.index for q in inst.qubits) for inst in layer if len(inst.qubits) == 2)
/tmp/ipykernel_26408/3986756806.py:8: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  inst_map = sorted([q.index for q in circuit.qubits])


Pauli bases chosen:  ['XXXX', 'YXYY', 'ZXZZ', 'XYXX', 'YYYY', 'ZYZZ', 'XZXX', 'YZYY', 'ZZZZ']
Model terms: ['IYIX', 'IXIX', 'IYYI', 'YXII', 'XYII', 'YIII', 'IXXI', 'ZIII', 'IYII', 'YZII', 'IIXI', 'IYXI', 'XZII', 'IZIX', 'IYZI', 'ZYII', 'XXII', 'IIIY', 'IZIZ', 'IZII', 'IIIX', 'YYII', 'IZZI', 'IXIZ', 'IZYI', 'IXYI', 'IXZI', 'IXIY', 'IXII', 'IIIZ', 'IYIZ', 'XIII', 'IZXI', 'IIZI', 'IIYI', 'ZZII', 'IZIY', 'ZXII', 'IYIY']
bases for singles:  [Pauli('IIXX'), Pauli('IIZY'), Pauli('IIYZ'), Pauli('IIZZ'), Pauli('IIYY'), Pauli('IIIY')]
Generated 330 circuits
Pauli bases chosen:  ['XXXX', 'YXYY', 'ZXZZ', 'XYXX', 'YYYY', 'ZYZZ', 'XZXX', 'YZYY', 'ZZZZ']
Model terms: ['IYIX', 'IXIX', 'IYYI', 'YXII', 'XYII', 'YIII', 'IXXI', 'ZIII', 'IYII', 'YZII', 'IIXI', 'IYXI', 'XZII', 'IZIX', 'IYZI', 'ZYII', 'XXII', 'IIIY', 'IZIZ', 'IZII', 'IIIX', 'YYII', 'IZZI', 'IXIZ', 'IZYI', 'IXYI', 'IXZI', 'IXIY', 'IXII', 'IIIZ', 'IYIZ', 'XIII', 'IZXI', 'IIZI', 'IIYI', 'ZZII', 'IZIY', 'ZXII', 'IYIY']
bases for singles:  [Pauli